# import libs

In [ ]:
import sys
import os
os.environ['TF_CPP_MIN_LOG_LEVEL'] = '3'

sys.path.append("/home/webphy/Desktop/dnn_processor/")  # just to enable `dataset`
sys.path.append(
    "/home/webphy/Desktop/dnn_processor/dataset/"
)  # just to enable `dataset`

import tensorflow as tf
import optuna
import dataset
import numpy as np
import keras
import joblib
from datetime import datetime
from keras import layers
import keras_cv

##  define constants and create dataset


In [ ]:
NTRIALS = 300
BATCH_SIZE = 64
EPOCHS = 100

train_images_rgx = "../../dataset/resized_dataset_texturas_v2/*/*.jpeg"
valid_images_rgx = "../../dataset/resized_dataset_texturas_v2_test/*/*.jpeg"

layer_options = ["Conv2D", "Depthwise2D", "Pointwise2D"]
activation_options = ["linear", "relu", "tanh"]
filter_size_options = [3, 4, 5, 8, 16]

train_dataset, val_dataset, num_classes = dataset.create_datasets(
    train_images_rgx, valid_images_rgx, BATCH_SIZE
)

## define create model function


In [ ]:
def clip_filter_size(fs, ih):
    return min(fs, ih)


def create_layer(trial, layer_type, layer_id, max_stride, input_h_size):
    if layer_type == "Conv2D":
        num_filters = trial.suggest_int(
            f"layer_{layer_id}_conv2d_filters", 4, 32, step=4
        )
        filter_size = trial.suggest_categorical(
            f"layer_{layer_id}_conv2d_fsize", filter_size_options
        )

        stride_size = trial.suggest_int(f"layer_{layer_id}_conv2d_ssize", 1, max_stride)
        layer = layers.Conv2D(
            num_filters, clip_filter_size(filter_size, input_h_size), stride_size
        )

    elif layer_type == "Depthwise2D":
        filter_size = trial.suggest_categorical(
            f"layer_{layer_id}_dwconv2d_fsize", filter_size_options
        )
        stride_size = trial.suggest_int(
            f"layer_{layer_id}_dwconv2d_ssize", 1, max_stride
        )
        layer = layers.DepthwiseConv2D(
            clip_filter_size(filter_size, input_h_size), stride_size
        )

    elif layer_type == "Pointwise2D":
        num_filters = trial.suggest_int(
            f"layer_{layer_id}_pwconv2d_filters", 4, 32, step=4
        )
        stride_size = trial.suggest_int(
            f"layer_{layer_id}_pwconv2d_ssize", 1, max_stride
        )
        layer = layers.Conv2D(num_filters, 1, stride_size)

    return layer


def create_augmentation_layers(trial, x):
    # augmix_severity = trial.suggest_float("augmix_severity", 0.0, 0.5)
    brightness_factor = trial.suggest_float("brightness_factor", 0, 0.2)
    gaussian_noise_factor = trial.suggest_float("gaussian_noise_factor", 0, 0.2)
    translate_factor = trial.suggest_float("translate_factor", 0, 0.2)

    # x = keras_cv.layers.AugMix(value_range=(0, 1), severity=augmix_severity)(x)
    x = layers.RandomBrightness(brightness_factor, (0, 1.0))(x)
    x = layers.RandomFlip()(x)
    x = layers.RandomTranslation(translate_factor, translate_factor)(x)
    x = layers.GaussianNoise(gaussian_noise_factor)(x)

    return x


def create_model(trial):
    # add the input layer
    x = x_in = layers.Input(shape=(256, 256, 3))

    x = create_augmentation_layers(trial, x)

    # suggent a number of layer
    num_layers = trial.suggest_categorical("num_layers", [1, 2, 4])

    # limits the stride
    max_stride = 16 // num_layers

    # convolutional layers
    for i in range(num_layers):
        layer_selected = trial.suggest_categorical(f"layer_{i}", layer_options)
        layer = create_layer(
            trial, layer_selected, i, max_stride, input_h_size=x.shape[1]
        )
        x = layer(x)
        act_selected = trial.suggest_categorical(f"activation_{i}", activation_options)
        x = layers.Activation(act_selected)(x)

    # output layer
    x = layers.Flatten()(x)
    x = layers.Dropout(trial.suggest_float("output_dropout", 0.3, 0.6))(x)
    x = layers.Dense(num_classes, activation="softmax")(x)

    return keras.Model(x_in, x)

# if testing


```
class MockTrial:
    def suggest_int(self, name, min, max, step=None, log=False):
        return np.random.randint(min, max)

    def suggest_float(self, name, min, max, step=None, log=False):
        return np.random.uniform(min, max)

    def suggest_categorical(self, name, options):
        print(name)
        return options[np.random.randint(0, len(options) - 1)]

    def suggest_discrete_uniform(self, name, min, max, q):
        return (np.random.randint(min, max) // q) * q

model = create_model(MockTrial())
model.build(input_shape=(BATCH_SIZE, 256, 256, 3))
model.summary()
model.save("saved_models/test/")
```

In [ ]:
def create_optimizer(trial):
    lr = trial.suggest_float("lr_inicial", 1e-4, 2e-3, log=True)
    optimizer = keras.optimizers.Adam(lr)
    return optimizer


def create_model_checkpointer(save_path):
    return keras.callbacks.ModelCheckpoint(
        save_path + "weights/",
        "val_accuracy",
        mode="max",
        save_best_only=True,
        save_weights_only=True,
    )


def crate_early_stopper():
    return keras.callbacks.EarlyStopping(
        monitor="val_accuracy",
        min_delta=0.1,
        patience=20,
        verbose=0,
        mode="max",
        start_from_epoch=50,
    )



def objective(trial):
    BATCH_SIZE = trial.suggest_int("batchsize", 32, 128, step=32)
    
    # load data
    train_dataset, val_dataset, num_classes = dataset.create_datasets(
        train_images_rgx, valid_images_rgx, BATCH_SIZE
    )

    # Build model and optimizer.
    model = create_model(trial)
    assert isinstance(model, keras.Model), "model is not a model"

    optimizer = create_optimizer(trial)

    # compile the model
    model.compile(
        optimizer, loss=keras.losses.categorical_crossentropy, metrics=["accuracy"]
    )

    # crate model checkpointer
    saved_best_model_path = f"saved_models/{EXP_ID}/best_model_t{trial._trial_id}/"
    model.save(saved_best_model_path, True, "tf")  # save all model

    model_chkp = create_model_checkpointer(saved_best_model_path)
    early_stopper = crate_early_stopper()

    # Training and validating cycle.
    history = model.fit(
        train_dataset,
        validation_data=val_dataset,
        epochs=EPOCHS,
        callbacks=[model_chkp, early_stopper],
        verbose="auto",
    )
    
    # load best model
    model.load_weights(saved_best_model_path + "weights/")    

    # Evaluate the model accuracy on the validation set.
    score = model.evaluate(val_dataset, verbose=0)
    
    # Return best model validation accuracy.
    return score[1]


def search():
    study = optuna.create_study(
        direction="maximize",
        sampler=optuna.samplers.TPESampler(),
    )
    study.optimize(objective, n_trials=NTRIALS)
    return study

# start search

In [ ]:
EXP_ID = "exp4_with_aug"
keras.utils.disable_interactive_logging()

# start searching for a model
study = search()

# save the study
now = datetime.now()
joblib.dump(study, "saved_studies/" + now.strftime("%Y:%m:%d:%H:%M:%S") + ".pkl")

# visualizations

In [ ]:
print("Number of finished trials: ", len(study.trials))

trial = study.best_trial
print(f"Best trial id: {trial._trial_id}")

print("  Value: ", trial.value)

print("  Params: ")
for key, value in trial.params.items():
    print("    {}: {}".format(key, value))

In [ ]:
from optuna.visualization import plot_optimization_history
from optuna.visualization import plot_parallel_coordinate
from optuna.visualization import plot_param_importances

In [ ]:
plot_optimization_history(study)

In [ ]:
plot_parallel_coordinate(
    study,
    [
        "lr_inicial",
        "num_layers",
        "layer_0",
        "activation_0",
    ],
)

In [ ]:
plot_param_importances(study)

In [ ]:
import keras

model = keras.models.load_model("/home/webphy/Desktop/dnn_processor/neural_network/automl/saved_models/exp2_with_aug/best_model_t185")

for l in model.layers:
    print(l.get_config())